<a href="https://colab.research.google.com/github/ancestor9/2025_Spring_Data-Management/blob/main/week_09/Text_Representation_and_Embedding_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 표현 기법과 임베딩
# **Data Representation**

## <font color='orange'>**4. Text data**
- **아래 그림을 이해하여야 한다.**
<img src='http://jalammar.github.io/images/numpy/numpy-nlp-embeddings.png'>

- **이 그림도 이해하여야 한다.**
<img src ='https://camo.githubusercontent.com/7fd0081ba6f27b73017f5307d663c8bf4e83bbd98e0eb148a34ffdd48be642f5/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f626f6f6b2e6b657261732e696f2f696d672f6368362f776f72645f656d62656464696e67732e706e67' width =400 height=400>

#### **2.6 사전학습 임베딩**
- Gensim / HuggingFace Transformers 사용 가능
- 예: word2vec-google-news-300
#### **2.6.1 [버트(Bidirectional Encoder Representations from Transformers, BERT)](https://wikidocs.net/115055)**

- **BERT라는 언어 모델의 토크나이저만 불러오는 것으로 토크나이저는 텍스트를 토큰(작은 단위)으로 분리하는 도구**

In [1]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 사전학습된 토크나이저

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [2]:
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [3]:
print(tokenizer.vocab['here'])

2182


In [4]:
print(tokenizer.vocab['embeddings'])

KeyError: 'embeddings'

In [6]:
print(tokenizer.vocab['##bed'])

8270


### BERT 기반 WordPiece Tokenizer는 다음과 같은 특성

- 토큰을 단어 단위로 자르지 않고 서브워드(subword) 단위로 자릅니다.
- 어휘집(tokenizer.vocab)에는 BERT 학습 시 사용된 서브워드만 포함됩니다.
- 새로운 단어는 ## 접두어가 붙은 서브워드 조각으로 분할됩니다

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = "Here is the sentence I want embeddings for."
tokens_wp = tokenizer.tokenize(sentence)
print("WordPiece 토큰:", tokens_wp)

# 각 토큰이 vocab에 있는지 확인하고 인덱스 출력
for token in tokens_wp:
    if token in tokenizer.vocab:
        print(f"{token} -> {tokenizer.vocab[token]}")
    else:
        print(f"{token} is not in vocab")


WordPiece 토큰: ['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']
here -> 2182
is -> 2003
the -> 1996
sentence -> 6251
i -> 1045
want -> 2215
em -> 7861
##bed -> 8270
##ding -> 4667
##s -> 2015
for -> 2005
. -> 1012


### 어휘집 크기를 작게 유지하면서도 새로운 단어를 처리할 수 있는 유연성 확보

#### **Tokenize By Sub-Word**
<img src ='https://testerstories.com/files/ai_learn/tokenizing-strategies.jpg'>

<img src ='https://testerstories.com/files/ai_learn/tokenize-middle-ground.jpg'>


**🧠 왜 이렇게 하나?**
- 단어 수를 제한(예: 30,522개)하면서도 유연하게 신조어, 희귀어 처리 가능
- 새로운 단어도 기존 subword 조합으로 처리 가능 → openaiGPTstyle도 잘 분해됨

In [8]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')


In [9]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


## **BERT**

In [10]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [11]:
# 0. 라이브러리 불러오기
import os
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# 1. wandb 비활성화
os.environ["WANDB_DISABLED"] = "true"


In [12]:
# 2. IMDb 데이터 로드 및 축소
raw_dataset = load_dataset("imdb")
dataset = raw_dataset["train"].shuffle(seed=42).select(range(100))  # train 데이터 중 100개 사용

dataset

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

In [13]:
# 3. 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 4. 토큰화 함수 정의
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
# 5. Train / Eval 분리
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 6. 모델 준비
# 사전학습된 BERT 모델의 가중치를 다운로드하고 불러옴
# 이전에 불러온 토크나이저와 동일한 "bert-base-uncased" 버전을 사용
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# 7. Trainer 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    # 'evaluation_strategy' is deprecated, use 'eval_strategy' instead.
    eval_strategy="epoch",
    logging_strategy="no",
    report_to="none"  # wandb 완전 끄기
)

# 8. Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [17]:

# 9. 학습 실행
trainer.train()

# 10. 예측 테스트
test_text = "The movie was absolutely fantastic!"
inputs = tokenizer(test_text, return_tensors="pt",
                   truncation=True,
                   padding=True,
                   max_length=256)

outputs = model(**inputs) # 입력 데이터의 모든 키-값 쌍을 모델에 전달

pred = torch.argmax(outputs.logits).item()
label = "긍정" if pred == 1 else "부정"
print(f"'{test_text}' → {label}")



Epoch,Training Loss,Validation Loss
1,No log,0.710584
2,No log,0.677196


'The movie was absolutely fantastic!' → 긍정


In [18]:
test_text = "The movie was absolutely terrible!"
inputs = tokenizer(test_text, return_tensors="pt",
                   truncation=True,
                   padding=True,
                   max_length=256)

outputs = model(**inputs) # 입력 데이터의 모든 키-값 쌍을 모델에 전달

pred = torch.argmax(outputs.logits).item()
label = "긍정" if pred == 1 else "부정"
print(f"'{test_text}' → {label}")

'The movie was absolutely terrible!' → 부정


## **HuggingFace**
- **DistilBERT(BERT의 경량화 버전)를 영화 리뷰 데이터셋(SST-2)에 파인튜닝한 것으로 아래 코드를 처음 실행할 때 이 모델이 자동으로 다운로드**

In [19]:
from transformers import pipeline

# 1. 감성 분석 파이프라인 생성
# distilbert-base-uncased-finetuned-sst-2-english --> default LLM
classifier = pipeline("sentiment-analysis")

# 2. 예측할 문장
text = "The professor explains very clearly and the class is enjoyable."

# 3. 예측
result = classifier(text)
result


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998881816864014}]

In [20]:
from transformers import pipeline

# 1. 다국어 감성 분석 모델 로드
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 2. 한국어 문장
texts = [
    "이 강의는 정말 유익하고 재미있었어요.",
    "교수님의 설명이 이해하기 어려웠습니다.",
    "수업이 별로였어요.",
    "내용이 알차고 좋았습니다.",
    "정말 시간 낭비였어요.",
    "이 수업 정말 미치도록 듣기 싫어, 다시는 듣고 싶지 않아"
]

# 3. 예측
for text in texts:
    result = classifier(text)[0]
    print(f"'{text}' → {result['label']} (score: {result['score']:.2f})")


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


'이 강의는 정말 유익하고 재미있었어요.' → 1 star (score: 0.40)
'교수님의 설명이 이해하기 어려웠습니다.' → 3 stars (score: 0.33)
'수업이 별로였어요.' → 2 stars (score: 0.27)
'내용이 알차고 좋았습니다.' → 4 stars (score: 0.37)
'정말 시간 낭비였어요.' → 3 stars (score: 0.25)
'이 수업 정말 미치도록 듣기 싫어, 다시는 듣고 싶지 않아' → 2 stars (score: 0.43)


In [21]:
! pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [30]:
# prompt: gradio로 챗봇화면을 만들어줘 classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")로 긍정과 부정을 보내주는 1star별 별 한 개로 그림으로 결과를 보야줘

import gradio as gr
from transformers import pipeline

# 감성 분석 파이프라인 로드
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def predict_sentiment(text):
    result = classifier(text)[0]
    label = result['label']
    score = result['score']

    # 별점으로 변환 (5점 만점)
    star_rating = int(round(score * 5))
    star_image = "⭐" * star_rating + "☆" * (5 - star_rating)

    return f"{label} ({star_image})"

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs="text",
    title="Sentiment Analysis with Star Rating",
    description="Enter some text, and the model will classify its sentiment and give a star rating!"
)

iface.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e8a43bdb0a8cc2233.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **LLM 사용 API 방식**

# **[Groq API](https://wikidocs.net/259655)**

- https://console.groq.com/playground
- https://python.langchain.com/docs/how_to/sequence/ **(Langchain Tutorial)**
- https://wikidocs.net/book/14314 **(한글판 랑체인 튜토리얼)**

In [22]:
from google.colab import userdata
groq_key = userdata.get('groq')

In [23]:
# LangChain과 Groq API를 연결하는 패키지.
# Groq은 초고속 LLM 서비스를 제공하는 AI 회사이며, 특히 LLaMA, Mixtral, Gemma 등의 모델을 빠르게 실행할 수 있음.
# 이 패키지를 사용하면 LangChain을 통해 Groq의 LLM을 손쉽게 활용할 수 있음.
%%capture
!pip install langchain-groq --quiet

In [24]:
from langchain_groq import ChatGroq

# ChatGroq 모델 초기화
llm = ChatGroq(
    model="gemma2-9b-it", # google/gemma-2-9b-it
    temperature=0.7,
    max_tokens=300,
    api_key=groq_key
)

In [25]:
llm.predict("안녕하세요?")

<ipython-input-25-e172f47a785f>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("안녕하세요?")


'안녕하세요! 👋  무엇을 도와드릴까요? 😊\n'

In [26]:
# 프롬프트 템플릿 정의
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절하고 유익한 AI 조수입니다. 한국의 역사와 문화에 대해 잘 알고 있습니다."),
    ("human", "{question}")
])

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절하고 유익한 AI 조수입니다. 한국의 역사와 문화에 대해 잘 알고 있습니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [27]:
# Chain 생성
chain = prompt | llm

In [28]:
# 질문 리스트
questions = [
    "한글의 창제 원리는 무엇인가요?",
    "김치의 역사와 문화적 중요성에 대해 설명해주세요.",
    "조선시대의 과거 제도에 대해 간단히 설명해주세요."
]

# 각 질문에 대한 답변 생성
for question in questions:
    response = chain.invoke({"question": question})
    print(f"질문: {question}")
    print(f"답변: {response.content}\n") # Use response.content to access the text
    print("*" * 150)

질문: 한글의 창제 원리는 무엇인가요?
답변: 네, 한국의 역사와 문화에 대한 질문을 받는 것을 좋아합니다!

한글의 창제 원리는 다음과 같습니다. 

세종대왕은 백성들이 쉽게 읽고 쓸 수 있는 문자를 만들고자 하셨습니다.  

* **음성 원리**: 한글은 
개별 소리 (음절)를 나타내는 글자를 사용합니다. 
한글 자음은 입술, 혀, 폐 등 **발음 기관의 모양**과 **움직임**을 기반으로 만들었고, 모음은 **음성이 만들어지는 공간**을 나타내는 데서 영감을 받았습니다. 
* **표현력**: 한글은 소리뿐만 아니라 **문맥과 의미**를 명확하게 표현하기 위해 
조합된 글자를 사용합니다. 
예를 들어, 자음과 모음을 합쳐 음절을 만들고, 여러 음절을 합쳐 단어를 만들 수 있습니다.
* **철학적 근거**:  한글은 
**자연의 순리**와 **인간의 언어 구조**를 반영하여 만들어졌습니다. 
세종대왕은 한글이 **모든 사람이 누구나 쉽게 이해하고 사용할 수 있는

******************************************************************************************************************************************************
질문: 김치의 역사와 문화적 중요성에 대해 설명해주세요.
답변: ## 김치: 한국의 역사와 문화를 담은 맛

김치는 단순한 음식을 넘어 한국의 역사와 문화를 대변하는 상징입니다. 

**역사**: 김치는 1천 년 이상 전부터 한국에서 만들어져 왔습니다. 고려 시대에는 이미 김치가 널리 즐겨 먹는 음식으로 자리를 잡았으며, 조선 시대에는 더욱 다양한 종류의 김치가 개발되었습니다. 

* **고대**: 초기 김치는 배, 무, 콩나물 등을 밑간으로 넣어 발효시킨 것으로 추정됩니다. 
* **고려**: 김치는 흔히 겨울철 식량으로 저장되었으며,  "유격"이라는 이름으로 불렸습니다. 
* **조선**: 깨, 마늘, 고추를 넣어 맵게 즐기는 김